In [49]:
st=[]
with open('stopwords.txt','r') as stopwords:
    for i in stopwords:
        st.append(i.strip())
#print(st)
with open("en-training.txt") as f: 
    data = f.readlines()
#print(data)
changed_data=[]
data_result=[]
data_list=[]
for txt in data[:]:
    lst=txt.strip().split()
    #print(lst)
    list1=[]
    complete_txt=""
    if(len(lst)>1):
        data_result.append(lst[-1].lower())
        for i in range(1,len(lst)-1):
            for j in lst[i].split(','):
                if j.lower() not in st and len(j)>1:
                    complete_txt=complete_txt+j.lower()+' '
                    list1.append(j.lower())
            #print(lst[i])
        changed_data.append(complete_txt)
        data_list.append(list1)
        #print(i)
#print(changed_data)
#print(data_result)
#print(data_list)
result_set=set(data_result)
res=list(result_set)
print(result_set)
#print(len(data_result))
#print(len(data_list))


{'complaint', 'bug', 'meaningless', 'comment', 'request', 'undefined'}


In [50]:
from collections import defaultdict
freqwords=defaultdict(int)
for words in data_list:
    for word in words:
        freqwords[word]=freqwords[word]+1
#freqwords
abc=freqwords.copy()
for j in abc:
    if freqwords[j]==1:
        freqwords.pop(j)
#freqwords
#print(freqwords)
filtdata=[]
for datas in data_list:
    filt=[]
    for word in datas:
        if freqwords[word]>1:
            filt.append(word)
    filtdata.append(filt)
#print(filtdata)

In [88]:
import gensim
dictbow=gensim.corpora.Dictionary(filtdata)
bowdata=[dictbow.doc2bow(mail) for mail in filtdata]
dense_bow = gensim.matutils.corpus2dense(bowdata, num_terms=len(dictbow)).transpose()
dense_bow.shape

tfidf=gensim.models.TfidfModel(bowdata)
records=tfidf[bowdata]
dense_tfidf=gensim.matutils.corpus2dense(records,num_terms=len(dictbow)).transpose()
dataset=dense_tfidf
#dataset.shape
target=[]
for i in data_result:
    if i == res[0]:
        target.append(5)
    if i == res[1]:
        target.append(4)
    if i == res[2]:
        target.append(0)
    if i == res[3]:
        target.append(3)
    if i == res[4]:
        target.append(1)
    else:
        target.append(2)
#print(target)
#print(dataset)

In [117]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10,random_state=None,shuffle=True)
kf.get_n_splits(dataset)
#print(kf)
testid=[]
trainid=[]

    #print(test_index,train_index)
#print("trainid")
#print(trainid)
#print("testid")
#print(testid)

In [121]:
from sklearn import svm # best  49.42 --->56.89
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB # 47.50  ---->51.96
from sklearn.naive_bayes import BernoulliNB  #44.40    --->49.34
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier    #43.88 --->5 nearest neighbours
from sklearn.naive_bayes import GaussianNB
#for i in trainid:
#    for j in i:
#        print (j)
#for i in range(0,len(trainid)):
 #   for j in trainid[i]:
  #      print(j)
from sklearn import tree

In [122]:
accuracies=[]
psArr=[]
rsArr=[]
farr=[]
for train_index,test_index in kf.split(dataset):
   # print("train ind {} test index {}".format(train_index,test_index))
    testid.append(test_index)
    trainid.append(train_index)
    train_set=[]
    train_tag=[]
    test_set=[]
    test_tag=[]
    for ind in train_index:
        train_set.append(dataset[ind])
        train_tag.append(target[ind])
    for ind in test_index:
        test_set.append(dataset[ind])
        test_tag.append(target[ind])
    clf = tree.DecisionTreeClassifier()
    clf.fit(train_set,train_tag)
    pred=clf.predict(test_set)
    incorrect=(pred!=test_tag).sum()
    accu=(len(test_set)-incorrect)/len(test_set)*100.0
    accuracies.append(accu)
    ps=precision_score(test_tag,pred,average="weighted")
    rs=recall_score(test_tag,pred,average="weighted")
    fscore=f1_score(test_tag,pred,average="weighted")
    farr.append(fscore)
    psArr.append(ps)
    rsArr.append(rs)
    print(accu,ps,rs,fscore)
print(max(accuracies))
print(max(psArr))
print(max(rsArr))
print(max(farr))   

34.527687296416936 0.32277727737836226 0.34527687296416937 0.33316852674979014


C:\Users\akash singh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\akash singh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


34.20195439739413 0.30622579685681406 0.34201954397394135 0.3215213534815529
37.45928338762215 0.3564717983884793 0.3745928338762215 0.36229221622117197
37.45928338762215 0.3272896801944754 0.3745928338762215 0.34619423659429843
38.762214983713356 0.36996233198541106 0.38762214983713356 0.37853131160046244
40.19607843137255 0.3727111214856313 0.40196078431372545 0.38516145249941197
36.60130718954248 0.3588077768105128 0.3660130718954248 0.3619468673458529
34.31372549019608 0.32327229867806057 0.3431372549019608 0.32916666881557505
32.35294117647059 0.2871501963682739 0.3235294117647059 0.3016428254811475
36.27450980392157 0.336168133002595 0.3627450980392157 0.3439208672870404
40.19607843137255
0.3727111214856313
0.40196078431372545
0.38516145249941197
